In [1]:
import pandas as pd
import seaborn as sns
import glob
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import probplot
from scipy.stats import zscore

In [2]:
sns.set_theme()

## Preprocessing

In [3]:
#load
fish_df = pd.read_csv('FW_Fish_count.csv')
fish_df.isnull().sum()

EVENT_DATE_YEAR     0
NO_OF_RUNS          0
SPECIES_NAME        0
LATIN_NAME         17
ALL_RUNS            0
EVENT_DATE          0
Average_count       0
dtype: int64

In [4]:
# Get a list of all csv files in the data directory 

files = glob.glob('data/*.csv')

In [5]:
#List to hold the DataFrame
dfs = []

#Loop through each file, read it into a DataFrame and skip the first row 
for file in files:
    i_df = pd.read_csv(file, encoding='utf-8', engine='python')
    dfs.append(i_df)
    
#Concatenate the DataFrames into a single DataFrame
pre_df = pd.concat(dfs, ignore_index=True)

In [6]:
pre_df

,@id,sample.samplingPoint,sample.samplingPoint.notation,sample.samplingPoint.label,sample.sampleDateTime,determinand.label,determinand.definition,determinand.notation,resultQualifier.notation,result,codedResultInterpretation.interpretation,determinand.unit.label,sample.sampledMaterialType.label,sample.isComplianceSample,sample.purpose.label,sample.samplingPoint.easting,sample.samplingPoint.northing
0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011624,APOLLO OFFICE UNITS RADCLIVE RD GAWCOTT,2022-03-18T08:51:00,Ammonia(N),Ammoniacal Nitrogen as N,111.0,NaN,5.10,NaN,mg/l,FINAL SEWAGE EFFLUENT,true,COMPLIANCE AUDIT (PERMIT),467993.0,232851.0
1,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011624,APOLLO OFFICE UNITS RADCLIVE RD GAWCOTT,2022-03-18T08:51:00,Oil & Grs Vs,"Visible oil or grease, significant trace: Pres...",664.0,NaN,0.00,NaN,pres/nf,FINAL SEWAGE EFFLUENT,true,COMPLIANCE AUDIT (PERMIT),467993.0,232851.0
2,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011624,APOLLO OFFICE UNITS RADCLIVE RD GAWCOTT,2022-03-18T08:51:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,NaN,9.70,NaN,mg/l,FINAL SEWAGE EFFLUENT,true,COMPLIANCE AUDIT (PERMIT),467993.0,232851.0
3,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011624,APOLLO OFFICE UNITS RADCLIVE RD GAWCOTT,2022-03-18T08:51:00,BOD ATU,BOD : 5 Day ATU,85.0,<,6.00,NaN,mg/l,FINAL SEWAGE EFFLUENT,true,COMPLIANCE AUDIT (PERMIT),467993.0,232851.0
4,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011624,APOLLO OFFICE UNITS RADCLIVE RD GAWCOTT,2022-04-07T14:21:00,NO FLOW/SAMP,No flow /No sample,7668.0,NaN,0.00,NaN,coded,FINAL SEWAGE EFFLUENT,true,COMPLIANCE AUDIT (PERMIT),467993.0,232851.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8217655,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,NE-49000209,HUMBER & TRIBS TO SPURN POINT - AD HOC,2018-12-18T11:30:00,Chromium -Cr,Chromium,3164.0,NaN,3.57,NaN,ug/l,RIVER / RUNNING SURFACE WATER,false,UNPLANNED REACTIVE MONITORING (POLLUTION INCID...,540049.0,410257.0
8217656,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,NE-49000209,HUMBER & TRIBS TO SPURN POINT - AD HOC,2018-12-18T11:30:00,Nickel - Ni,Nickel,6462.0,NaN,7.48,NaN,ug/l,RIVER / RUNNING SURFACE WATER,false,UNPLANNED REACTIVE MONITORING (POLLUTION INCID...,540049.0,410257.0
8217657,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,NE-49000209,HUMBER & TRIBS TO SPURN POINT - AD HOC,2018-12-18T11:30:00,Calcium - Ca,Calcium,241.0,NaN,149.00,NaN,mg/l,RIVER / RUNNING SURFACE WATER,false,UNPLANNED REACTIVE MONITORING (POLLUTION INCID...,540049.0,410257.0
8217658,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,NE-49000209,HUMBER & TRIBS TO SPURN POINT - AD HOC,2018-12-18T11:30:00,Aluminium-Al,Aluminium,6057.0,NaN,1010.00,NaN,ug/l,RIVER / RUNNING SURFACE WATER,false,UNPLANNED REACTIVE MONITORING (POLLUTION INCID...,540049.0,410257.0


In [7]:
#pivot determinants 
df_wide = pre_df.pivot_table(index=['sample.samplingPoint.label','sample.sampleDateTime','sample.samplingPoint.easting','sample.samplingPoint.northing'],columns='determinand.label',
                           
                           values='result'
                           )

In [8]:
# List of column names to keep
cols_to_keep = ['Alky pH 4.5','Chloride Ion','Orthophospht','Sulphate SO4','Ammonia(N)','TurbidityNTU','Temp Water','Cond @ 25C','Oxygen Diss', 'pH','BOD ATU','COD as O2','Nitrate-N','Nitrite-N','Hardness','SALinsitu']

# Select only the desired columns
df_wide = df_wide[cols_to_keep]

In [9]:
#convert the index columns to dataset columns
df_wide_index = df_wide.reset_index().rename(columns={'index': 'sample.sampleDateTime'})

In [10]:
#drop columns that are not used in the study as decided by researcher based on literatures
df_wide_index = df_wide_index.drop(columns=['sample.samplingPoint.label','sample.samplingPoint.easting','sample.samplingPoint.northing'])

In [11]:
#rename columns to readable names

df_renamed = df_wide_index.rename(
    columns={
       'sample.sampleDateTime': 'Date'
        
    })


#### NOTE
Run the cell below only once and continue working with the clean csv file

In [12]:
#Write the dataframe to a CSV file
df_renamed.to_csv('freshwater.csv', index=False)

## Data analysis and visualization

In [13]:
#read clean csv to a new dataframe df
df = pd.read_csv('freshwater.csv', encoding='utf-8', engine='python')

In [14]:
df

,Date,Alky pH 4.5,Chloride Ion,Orthophospht,Sulphate SO4,Ammonia(N),TurbidityNTU,Temp Water,Cond @ 25C,Oxygen Diss,pH,BOD ATU,COD as O2,Nitrate-N,Nitrite-N,Hardness,SALinsitu
0,2016-10-27T13:00:00,NaN,2320.0,0.01,NaN,0.772,4.0,14.5,10305.0,NaN,7.9,2.75,50.0,NaN,NaN,NaN,NaN
1,2018-05-22T13:53:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-06-22T13:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-06-25T10:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-06-29T12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825690,2021-10-11T10:49:00,NaN,NaN,NaN,NaN,7.800,NaN,NaN,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN,NaN
825691,2022-01-04T11:00:00,NaN,NaN,NaN,NaN,3.300,NaN,NaN,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN
825692,2022-04-21T11:07:00,NaN,NaN,NaN,NaN,28.000,NaN,NaN,NaN,NaN,NaN,3.70,NaN,NaN,NaN,NaN,NaN
825693,2022-07-06T14:39:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df['Date'] = pd.to_datetime(df['Date']) 
#change date column format to date format

In [16]:
# Replacing string NAN values with actual NAN value (np.nan)
def convert_to_nan(df):
    n_col = df.shape[1]
    for index in range(n_col):
        df.iloc[:, index]  = df.iloc[:, index].replace("NAN", np.nan)
    return df

df = convert_to_nan(df)

/tmp/ipykernel_2704833/3762375366.py:5: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, index]  = df.iloc[:, index].replace("NAN", np.nan)


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825695 entries, 0 to 825694
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Date          825695 non-null  datetime64[ns]
 1   Alky pH 4.5   172953 non-null  float64       
 2   Chloride Ion  102957 non-null  float64       
 3   Orthophospht  232501 non-null  float64       
 4   Sulphate SO4  15687 non-null   float64       
 5   Ammonia(N)    429605 non-null  float64       
 6   TurbidityNTU  32868 non-null   float64       
 7   Temp Water    347867 non-null  float64       
 8   Cond @ 25C    201914 non-null  float64       
 9   Oxygen Diss   207224 non-null  float64       
 10  pH            243978 non-null  float64       
 11  BOD ATU       448448 non-null  float64       
 12  COD as O2     176685 non-null  float64       
 13  Nitrate-N     212194 non-null  float64       
 14  Nitrite-N     224691 non-null  float64       
 15  Hardness      262

In [1]:
fish_df['EVENT_DATE'] = pd.to_datetime(fish_df['EVENT_DATE'],dayfirst=True)

#change date column format to date format

NameError: name 'pd' is not defined

In [19]:
# group fih data by month and aggregate non-NaN values
# fish_grouped_df = fish_df.groupby(pd.Grouper(key='EVENT_DATE', freq='D')).agg(lambda x: x.dropna().mean())
fish_grouped_df = fish_df.groupby("EVENT_DATE").agg(lambda x: x.mean())


fish_grouped_df = fish_grouped_df.reset_index().rename(columns={'index': 'Date'})
fish_grouped_df["Date"] = fish_grouped_df["EVENT_DATE"]

/tmp/ipykernel_2704833/4286079134.py:3: FutureWarning: ['SPECIES_NAME', 'LATIN_NAME'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  fish_grouped_df = fish_df.groupby("EVENT_DATE").agg(lambda x: x.mean())


In [28]:
fish_grouped_df
fish_grouped_df.isnull().sum()

EVENT_DATE         0
EVENT_DATE_YEAR    0
NO_OF_RUNS         0
ALL_RUNS           0
Average_count      0
Date               0
dtype: int64

In [21]:
fish = fish_grouped_df['Average_count']

In [35]:
# group by month and aggregate non-NaN values
grouped_df = df.groupby(pd.Grouper(key='Date', freq='D')).agg(lambda x: x.dropna().mean())

grouped_df = grouped_df.reset_index().rename(columns={'index': 'Date'})
grouped_df.shape

(4017, 17)

In [39]:
grouped_df

,Date,Alky pH 4.5,Chloride Ion,Orthophospht,Sulphate SO4,Ammonia(N),TurbidityNTU,Temp Water,Cond @ 25C,Oxygen Diss,pH,BOD ATU,COD as O2,Nitrate-N,Nitrite-N,Hardness,SALinsitu
0,2012-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-01-03,167.181818,104.144444,1.405902,47.250000,3.378989,43.416667,8.591707,992.817391,9.519000,7.815676,8.117105,70.200000,21.200421,0.270746,270.0,13.736667
3,2012-01-04,193.600000,122.520879,0.547661,89.083333,2.213170,7.594118,6.921075,1301.784946,9.002637,7.729412,37.850899,174.556436,10.337000,0.326638,307.2,20.572222
4,2012-01-05,181.490909,113.369841,0.504410,71.920000,1.184673,14.750000,7.390233,1087.367089,10.031494,7.860330,16.172447,140.811111,10.844410,0.068047,283.5,14.195000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4012,2022-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4013,2022-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4014,2022-12-28,188.259259,98.333333,0.135548,NaN,1.182788,NaN,7.396774,743.827586,11.094828,7.892667,5.683333,NaN,10.003290,0.053148,NaN,NaN
4015,2022-12-29,179.307692,NaN,0.070077,NaN,0.667857,NaN,7.061538,681.833333,11.266667,8.047692,3.150000,NaN,9.229231,0.038308,NaN,32.080000


### Removing outliers

In [34]:
#df_numeric = grouped_df.drop(columns=['Date'])

#### Using Interquatile range (IQR) method

In [40]:
# define threshold multiplier for the IQR method
threshold_multiplier = 1.5

# create a new dataframe to store the cleaned data
cleaned_df = pd.DataFrame()

# iterate over each column of the original dataframe
for col in grouped_df.columns:
    # calculate the IQR for the column
    Q1 = grouped_df[col].quantile(0.25)
    Q3 = grouped_df[col].quantile(0.75)
    IQR = Q3 - Q1
    # identify outliers using the IQR method
    lower_bound = Q1 - threshold_multiplier * IQR
    upper_bound = Q3 + threshold_multiplier * IQR
    outliers = (grouped_df[col] < lower_bound) | (grouped_df[col] > upper_bound)
    # create a new column with outliers removed
    cleaned_col = grouped_df[col][~outliers]
    # add the cleaned column to the new dataframe
    cleaned_df[col] = cleaned_col

# print the cleaned dataframe
cleaned_df.describe()

,Alky pH 4.5,Chloride Ion,Orthophospht,Sulphate SO4,Ammonia(N),TurbidityNTU,Temp Water,Cond @ 25C,Oxygen Diss,pH,BOD ATU,COD as O2,Nitrate-N,Nitrite-N,Hardness,SALinsitu
count,2564.000000,2878.000000,2707.000000,2237.000000,3273.000000,2162.000000,3576.000000,2515.000000,2877.000000,3033.000000,3531.000000,3398.000000,2589.000000,2676.000000,2340.000000,2530.000000
mean,199.056464,293.583226,0.355535,344.916505,1.536333,10.465426,12.027007,1098.476210,9.642174,7.859075,41.595983,256.965330,8.106355,0.059721,308.300828,21.514931
std,29.563270,268.488686,0.217196,449.550295,0.903339,8.008014,4.150598,425.043952,1.437990,0.151510,26.700273,110.872205,2.562186,0.031419,88.030988,11.501550
min,115.473684,7.400000,0.010000,10.000000,0.030000,1.000000,0.833333,25.860000,5.277000,7.421765,1.000000,10.000000,0.900000,0.004000,77.057143,0.000000
25%,179.322115,89.905055,0.192139,63.740000,0.872964,4.793182,8.513541,821.440959,8.507519,7.777348,20.981905,188.073722,6.335750,0.037545,253.558059,14.304643
50%,200.718845,186.300909,0.316562,121.812500,1.311793,7.700000,12.002261,988.335227,9.514415,7.873978,39.616875,268.673248,8.048806,0.052512,308.575000,25.203368
75%,220.149756,423.473227,0.481252,453.225000,1.965541,13.391422,15.623501,1274.607375,10.850000,7.958934,57.488067,332.984302,9.666010,0.076672,364.760417,31.316250
max,283.292683,1232.364706,1.047242,1978.000000,4.590874,38.541176,23.900000,2595.916667,13.700000,8.265000,123.100000,566.769231,15.810000,0.163400,543.250000,36.001667


In [46]:
cleaned_df.isnull().sum()

Date               0
Alky pH 4.5     1453
Chloride Ion    1139
Orthophospht    1310
Sulphate SO4    1780
Ammonia(N)       744
TurbidityNTU    1855
Temp Water       441
Cond @ 25C      1502
Oxygen Diss     1140
pH               984
BOD ATU          486
COD as O2        619
Nitrate-N       1428
Nitrite-N       1341
Hardness        1677
SALinsitu       1487
dtype: int64

In [42]:
full_df = cleaned_df.set_index("Date").join(fish_grouped_df.set_index("Date"), on="Date", how = "outer")

In [43]:
full_df.isnull().sum()

Alky pH 4.5        1453
Chloride Ion       1139
Orthophospht       1310
Sulphate SO4       1780
Ammonia(N)          744
TurbidityNTU       1855
Temp Water          441
Cond @ 25C         1502
Oxygen Diss        1140
pH                  984
BOD ATU             486
COD as O2           619
Nitrate-N          1428
Nitrite-N          1341
Hardness           1677
SALinsitu          1487
EVENT_DATE         1991
EVENT_DATE_YEAR    1991
NO_OF_RUNS         1991
ALL_RUNS           1991
Average_count      1991
dtype: int64

In [48]:
full_df = full_df[full_df['Average_count'].notna()]

In [51]:
full_df = full_df.fillna(full_df.mean())

In [53]:
percent_missing = full_df.isnull().sum() * 100 / len(full_df)
missing_value_df = pd.DataFrame({'column_name': full_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
Alky pH 4.5,Alky pH 4.5,0.0
Chloride Ion,Chloride Ion,0.0
Orthophospht,Orthophospht,0.0
Sulphate SO4,Sulphate SO4,0.0
Ammonia(N),Ammonia(N),0.0
TurbidityNTU,TurbidityNTU,0.0
Temp Water,Temp Water,0.0
Cond @ 25C,Cond @ 25C,0.0
Oxygen Diss,Oxygen Diss,0.0
pH,pH,0.0


In [55]:
full_df

,Alky pH 4.5,Chloride Ion,Orthophospht,Sulphate SO4,Ammonia(N),TurbidityNTU,Temp Water,Cond @ 25C,Oxygen Diss,pH,BOD ATU,COD as O2,Nitrate-N,Nitrite-N,Hardness,SALinsitu,ALL_RUNS,Average_count
Date,,,,,,,,,,,,,,,,,,
2012-01-02,202.702330,290.161737,0.395029,287.018100,1.550881,10.138017,12.882616,1115.134319,9.369837,7.887049,41.806727,259.462151,8.194472,0.063292,313.025758,22.524549,1.000000,1.000000
2012-01-04,193.600000,122.520879,0.547661,89.083333,2.213170,7.594118,6.921075,1301.784946,9.002637,7.729412,37.850899,174.556436,10.337000,0.063292,307.200000,20.572222,2.000000,2.000000
2012-01-05,181.490909,113.369841,0.504410,71.920000,1.184673,14.750000,7.390233,1087.367089,10.031494,7.860330,16.172447,140.811111,10.844410,0.068047,283.500000,14.195000,1.333333,1.333333
2012-01-06,225.589744,76.544444,0.380976,131.500000,0.650909,10.138017,6.379655,824.392308,10.865500,7.825333,39.554936,174.591667,9.376547,0.052374,318.000000,20.026667,4.647059,2.647059
2012-01-08,202.702330,290.161737,0.395029,287.018100,1.550881,10.138017,6.610000,1115.134319,9.369837,7.887049,41.806727,259.462151,8.194472,0.063292,313.025758,22.524549,29.357143,21.373016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-05,189.292035,213.972131,0.120698,196.333333,0.512113,10.138017,7.581928,756.168067,10.324298,7.801773,33.246512,267.452381,7.772568,0.050829,208.888889,23.420000,37.166667,18.750000
2022-12-07,182.768519,181.815909,0.126429,287.018100,0.642478,10.138017,5.316471,765.852113,11.337664,7.916333,57.481250,323.219512,8.172836,0.031036,313.025758,28.695714,32.000000,24.339286
2022-12-08,181.613208,71.016327,0.128698,287.018100,0.768546,10.138017,4.368175,696.209091,12.096786,7.933411,46.443605,261.861538,8.892500,0.036092,313.025758,30.445000,19.277778,17.055556


In [56]:
#Write the dataframe to a CSV file
full_df = full_df.reset_index()
full_df.to_csv('full_data.csv', index=False)

In [57]:
full_df

,Date,Alky pH 4.5,Chloride Ion,Orthophospht,Sulphate SO4,Ammonia(N),TurbidityNTU,Temp Water,Cond @ 25C,Oxygen Diss,pH,BOD ATU,COD as O2,Nitrate-N,Nitrite-N,Hardness,SALinsitu,ALL_RUNS,Average_count
0,2012-01-02,202.702330,290.161737,0.395029,287.018100,1.550881,10.138017,12.882616,1115.134319,9.369837,7.887049,41.806727,259.462151,8.194472,0.063292,313.025758,22.524549,1.000000,1.000000
1,2012-01-04,193.600000,122.520879,0.547661,89.083333,2.213170,7.594118,6.921075,1301.784946,9.002637,7.729412,37.850899,174.556436,10.337000,0.063292,307.200000,20.572222,2.000000,2.000000
2,2012-01-05,181.490909,113.369841,0.504410,71.920000,1.184673,14.750000,7.390233,1087.367089,10.031494,7.860330,16.172447,140.811111,10.844410,0.068047,283.500000,14.195000,1.333333,1.333333
3,2012-01-06,225.589744,76.544444,0.380976,131.500000,0.650909,10.138017,6.379655,824.392308,10.865500,7.825333,39.554936,174.591667,9.376547,0.052374,318.000000,20.026667,4.647059,2.647059
4,2012-01-08,202.702330,290.161737,0.395029,287.018100,1.550881,10.138017,6.610000,1115.134319,9.369837,7.887049,41.806727,259.462151,8.194472,0.063292,313.025758,22.524549,29.357143,21.373016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,2022-12-05,189.292035,213.972131,0.120698,196.333333,0.512113,10.138017,7.581928,756.168067,10.324298,7.801773,33.246512,267.452381,7.772568,0.050829,208.888889,23.420000,37.166667,18.750000
2022,2022-12-07,182.768519,181.815909,0.126429,287.018100,0.642478,10.138017,5.316471,765.852113,11.337664,7.916333,57.481250,323.219512,8.172836,0.031036,313.025758,28.695714,32.000000,24.339286
2023,2022-12-08,181.613208,71.016327,0.128698,287.018100,0.768546,10.138017,4.368175,696.209091,12.096786,7.933411,46.443605,261.861538,8.892500,0.036092,313.025758,30.445000,19.277778,17.055556
2024,2022-12-09,180.666667,35.100000,0.200904,287.018100,1.782130,10.138017,3.596154,714.621212,12.220563,7.990278,26.866667,302.357143,7.085451,0.033739,248.000000,20.183333,44.763158,43.473684


In [62]:
new = full_df.head(5)
new
new.to_csv('head.csv')